In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
cd /gdrive/MyDrive/dacon/handSignal/

/gdrive/MyDrive/dacon/handSignal


In [ ]:
!rm -r ./save_weights/effient_*

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9810569973762879850, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16185556992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12238099409443257790
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
!pip install imgaug --upgrade

     |████████████████████████████████| 948 kB 5.0 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [ ]:
# GPU 환경 설정
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 파일경로 설정
import json

# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
import cv2
from tqdm import tqdm

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [ ]:
'''Generator'''
from tensorflow.keras.utils import Sequence
from google.colab.patches import cv2_imshow
from imgaug import augmenters as iaa
from imgaug.augmentables.batches import UnnormalizedBatch
import random

class Generator(Sequence):
    def __init__(self, src_path, input_shape, batch_size, frame_size=30, augs=None):
        new_image_directory = src_path + '/new_images'
        new_train_image_directory = new_image_directory + '/train'

        action_information = pd.read_csv(src_path + '/action_information.csv')

        classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()
        
        new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))

        train_answer = []
        train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('_')[-1]))
        for train_image_directory in train_image_directories : 
            json_path = glob(train_image_directory + '/*.json')[0]
            js = json.load(open(json_path))
            action = js.get('action')
            train_answer.append(action)
        
        self.class_num = len(classes)
        self.x_data = []
        self.y_data = []
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.augs = iaa.Sequential(augs)
        self.frame_size = frame_size

        for new_train_image_directory, action in tqdm(zip(new_train_image_directories, train_answer), total = len(new_train_image_directories)) : 
            image_paths = sorted(glob(new_train_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
            image_len = len(image_paths)

            for i in range(0, image_len, frame_size):
                datas = image_paths[i:(i+frame_size)]
                if len(datas) == frame_size:
                    self.x_data.append(datas)
                    self.y_data.append(classes[action])
        self.on_epoch_end()
        print("Generator Initialized!!")
    
    def __len__(self):
        return round(len(self.x_data) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx * self.batch_size:(idx+1)*self.batch_size]
        
        for batch_i in batch_index:
            batch_x.append(self.x_data[batch_i])
            batch_y.append(self.y_data[batch_i])
        
        out_x, out_y = self.data_gen(batch_x, batch_y)
        return out_x, out_y

    def on_epoch_end(self):
        self.index = np.arange(len(self.x_data))
        np.random.shuffle(self.index)
    
    def data_gen(self, x ,y):
        input_x = np.zeros((self.batch_size, self.frame_size, self.input_shape[0], self.input_shape[1], 3), dtype=np.float)
        input_y = np.zeros((self.batch_size, self.class_num), dtype=np.float)

        frame_imgs = []

        for i, frame_x in enumerate(x): # get 30 frames img paths
            imgs = []
            for j in range(len(frame_x)):
                img = cv2.imread(frame_x[j])
                imgs.append(cv2.resize(img, (self.input_shape[0], self.input_shape[1])))
            batch_imgs = UnnormalizedBatch(images=imgs, data=y[i])
            batch_aug_imgs = list(self.augs.augment_batches(batches=batch_imgs))

            for j in range(len(frame_x)):
                aug_img = batch_aug_imgs[0].images_aug[j]
                input_x[i][j] = aug_img.astype(np.float) / 255.0

            input_y[i] = y[i]
        return input_x, input_y
        


In [ ]:
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 1
input_shape = (224, 224)

train_gen = Generator(src_path, input_shape, batch_size, augs=[])
x, y = train_gen.__getitem__(0)
print(x.shape)
print(y.shape)

100%|██████████| 142/142 [00:00<00:00, 406.47it/s]


Generator Initialized!!
(1, 30, 224, 224, 3)
(1, 6)


# build model

In [ ]:

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L2

class FastSlowNetwork:
    def __init__(self, input_shape, alpha=8, fast_stride=2, layers=[3, 4, 6, 3], classes_num=6, weight_decay=1e-7):
        self.input_shape = input_shape
        self.l2_reg = L2(weight_decay)
        self.layers = layers
        self.classes_num = classes_num
        self.fast_temporal_stride = fast_stride
        self.slow_temporal_stride = self.fast_temporal_stride * alpha
        
        self.slow_start_ch = 64
        self.fast_start_ch = self.slow_start_ch / alpha

    def conv3d_bn(self, x, filters, kernel_size=3, strides=(1,1,1), padding="same"):
        out = Conv3D(filters, kernel_size, strides, padding, kernel_regularizer=self.l2_reg, use_bias=False)(x)
        out = BatchNormalization()(out)
        out = ReLU()(out)
        return out
    
    def bottleneck(self, x, filters, kernel_size, strides, first_layer=False):
        identity = x
        out = self.conv3d_bn(x, filters, kernel_size, (1, 1, 1), padding="same")
        out = self.conv3d_bn(out, filters, (1, 3, 3), strides, padding="same")
        out = self.conv3d_bn(out, filters*4, (1, 1, 1), (1, 1, 1), padding="same")
        
        if strides[1] == 2 or first_layer:
            identity = self.conv3d_bn(identity, filters*4, (1, 1, 1), strides, padding="same")
        
        return Add()([identity, out])

    def slow_path(self, x, laterals):
        out = self.conv3d_bn(x, self.slow_start_ch, kernel_size=(1, 7, 7), strides=(1, 2, 2), padding="same")
        out = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding="same")(out)
        out = Concatenate()([out, laterals[0]])

        '''first block'''
        for i in range(self.layers[0]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.slow_start_ch, kernel_size=(1, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[0] - 1: # last layer
                out = self.bottleneck(out, self.slow_start_ch, kernel_size=(1, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.slow_start_ch, kernel_size=(1, 1, 1), strides=(1, 1, 1), first_layer=False )
        out = Concatenate()([out, laterals[1]])

        '''second block'''
        for i in range(self.layers[1]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.slow_start_ch*2, kernel_size=(1, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[1] - 1: # last layer
                out = self.bottleneck(out, self.slow_start_ch*2, kernel_size=(1, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.slow_start_ch*2, kernel_size=(1, 1, 1), strides=(1, 1, 1), first_layer=False )
        out = Concatenate()([out, laterals[2]])

        '''third block'''
        for i in range(self.layers[2]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.slow_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[2] - 1: # last layer
                out = self.bottleneck(out, self.slow_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.slow_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        out = Concatenate()([out, laterals[3]])

        '''fourth block'''
        for i in range(self.layers[3]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.slow_start_ch*8, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            else:
                out = self.bottleneck(out, self.slow_start_ch*8, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        
        out = GlobalAveragePooling3D()(out)
        return out

    def fast_path(self, x):
        out = self.conv3d_bn(x, self.fast_start_ch, kernel_size=(5, 7, 7), strides=(1, 2, 2), padding="same")
        out = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding="same")(out)
        lateral1 = Conv3D(self.fast_start_ch * 2, (5, 1, 1), strides=(8, 1, 1), use_bias=False, padding="same")(out)

        '''first block'''
        for i in range(self.layers[0]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.fast_start_ch, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[0] - 1: # last layer
                out = self.bottleneck(out, self.fast_start_ch, kernel_size=(3, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.fast_start_ch, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        lateral2 = Conv3D(self.fast_start_ch*8, (5, 1, 1), strides=(8, 1, 1), use_bias=False, padding="same")(out)

        '''second block'''
        for i in range(self.layers[1]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.fast_start_ch*2, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[1] - 1: # last layer
                out = self.bottleneck(out, self.fast_start_ch*2, kernel_size=(3, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.fast_start_ch*2, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        lateral3 = Conv3D(self.fast_start_ch*16, (5, 1, 1), strides=(8, 1, 1), use_bias=False, padding="same")(out)

        '''third block'''
        for i in range(self.layers[2]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.fast_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            elif i == self.layers[2] - 1: # last layer
                out = self.bottleneck(out, self.fast_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 2, 2), first_layer=False )
            else:
                out = self.bottleneck(out, self.fast_start_ch*4, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        lateral4 = Conv3D(self.fast_start_ch*32, (5, 1, 1), strides=(8, 1, 1), use_bias=False, padding="same")(out)
        
        '''fourth block'''
        for i in range(self.layers[3]):
            if i ==0: # first layer
                out = self.bottleneck(out, self.fast_start_ch*8, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=True )
            else:
                out = self.bottleneck(out, self.fast_start_ch*8, kernel_size=(3, 1, 1), strides=(1, 1, 1), first_layer=False )
        out = GlobalAveragePooling3D()(out)
        return out, [lateral1, lateral2, lateral3, lateral4]

    def buildModel(self):
        model_input = Input(shape = self.input_shape)

        '''fast_path'''
        fast_input = Lambda(lambda x : x[:, ::self.fast_temporal_stride, :, :, :])(model_input)
        fast_out, laterals = self.fast_path(fast_input)

        '''slow_path'''
        slow_input = Lambda(lambda x : x[:, ::self.slow_temporal_stride, :, :, :])(model_input)
        slow_out = self.slow_path(slow_input, laterals)

        out = Concatenate()([fast_out, slow_out])
        out = Dropout(0.5)(out)
        model_out = Dense(units=self.classes_num, activation="softmax")(out)
        model = Model(inputs=model_input, outputs=[model_out])
        return model

## Model Test

In [ ]:
fsn = FastSlowNetwork(input_shape=(24, 224, 224, 3))
model = fsn.buildModel()
# model.save("/gdrive/MyDrive/dacon/handSignal/test.h5")
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 24, 224, 224 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 24, 224, 224, 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_116 (Conv3D)             (None, 24, 112, 112, 5880        lambda_2[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 3, 224, 224,  0           input_2[0][0]                    
____________________________________________________________________________________________

# Training

In [ ]:
import imgaug.augmenters as iaa

augs =[
    #    iaa.Fliplr(0.5),
       
        # iaa.Sometimes(0.2,iaa.Sharpen()),
       
        iaa.SomeOf((0,2),[
            iaa.MultiplyAndAddToBrightness(),
            iaa.GammaContrast()
        ]),

        # iaa.SomeOf((0,1), [
        #     iaa.Sometimes(0.7, iaa.AdditiveGaussianNoise()),
        # ]),

        # iaa.SomeOf((0,2),[
        #     iaa.ScaleX((0.8, 1.2)),
        #     iaa.ScaleY((0.8, 1.2)),
        # ]),

]

In [ ]:
# Modeling
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

'''Train options'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 8
input_shape = (224, 224)
epochs = 2000
lr = 1e-3
frame_size = 32
fast_stride = 2

model_name = "slow_fast_v2"
filename = src_path+"/save_weights/%s_{epoch:05d}.h5"%(model_name)

callback = [ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5),
            ModelCheckpoint(filename, monitor="loss", save_weights_only=True)]

'''Build Model'''
with tf.device('/device:GPU:0'):
    fsn = FastSlowNetwork(input_shape=(frame_size, input_shape[0], input_shape[1], 3), fast_stride=fast_stride)
    model = fsn.buildModel()
    # model.load_weights(src_path+"/save_weights/effient_00007.h5")
    model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=lr, momentum=0.9), metrics=['accuracy'])

    '''Train'''
    train_gen = Generator(src_path, input_shape, batch_size, frame_size=frame_size, augs=[])
    # val_gen = Generator(src_path, input_shape, batch_size, augs=[], is_train=False)
    model.fit_generator(train_gen, epochs=epochs, max_queue_size=50, workers=32, callbacks=callback)

100%|██████████| 142/142 [00:00<00:00, 367.37it/s]


Generator Initialized!!
Epoch 1/2000
77/77 [==============================] - 74s 492ms/step - loss: 4.8219 - accuracy: 0.1737
Epoch 2/2000
77/77 [==============================] - 71s 526ms/step - loss: 4.6242 - accuracy: 0.1834
Epoch 3/2000
77/77 [==============================] - 69s 528ms/step - loss: 4.2564 - accuracy: 0.1948
Epoch 4/2000
77/77 [==============================] - 69s 534ms/step - loss: 3.8728 - accuracy: 0.1867
Epoch 5/2000
77/77 [==============================] - 68s 519ms/step - loss: 3.9933 - accuracy: 0.1948
Epoch 6/2000
77/77 [==============================] - 64s 527ms/step - loss: 3.3463 - accuracy: 0.2175
Epoch 7/2000
77/77 [==============================] - 59s 526ms/step - loss: 3.3313 - accuracy: 0.2321
Epoch 8/2000
77/77 [==============================] - 69s 527ms/step - loss: 3.2054 - accuracy: 0.1916
Epoch 9/2000
77/77 [==============================] - 69s 525ms/step - loss: 2.9376 - accuracy: 0.2289
Epoch 10/2000
77/77 [============================

In [ ]:
'''for test generator'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 1
input_shape = (224, 224)
augs= [
       iaa.Fliplr(1.0)
]
gen = Generator(src_path, input_shape, batch_size, augs=augs, is_train=True)
# gen.__getitem__(0)

100%|██████████| 142/142 [00:00<00:00, 154.70it/s]


19993
[ 5774  2301 11790 ... 11940 13295  6158]
Generator Initialized!!


# Prediction

In [ ]:
import numpy as np
from glob import glob
src_path = '/gdrive/MyDrive/dacon/handSignal'
frame_size = 32
input_shape = (224, 224)
fast_strides = 2
new_test_image_directory = src_path + '/new_images/test'
new_test_image_directories = sorted(glob(new_test_image_directory + '/*'), key = lambda x : int(x.split('file_')[-1]))
fsn = FastSlowNetwork(input_shape=(frame_size, input_shape[0], input_shape[1], 3), fast_stride=fast_strides)
model = fsn.buildModel()
model.load_weights(src_path+"/save_weights/slow_fast_v2_00043.h5")

predictions = []
for new_test_image_directory in tqdm(new_test_image_directories, total = len(new_test_image_directories)) :
    image_paths = sorted(glob(new_test_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    
    batch_test_images = []
    for i in range(0, image_len, frame_size):
        datas = image_paths[i:(i+frame_size)]
        if len(datas) != frame_size:
            break
        test_images  = []
        for data in datas:
            img = cv2.imread(data)
            img = cv2.resize(img, (224, 224))
            img = img/255
            test_images.append(img)
        batch_test_images.append(test_images)
    print(np.array(batch_test_images).shape)
    prediction = np.mean(model.predict(np.array(batch_test_images)), axis = 0)
    predictions.append(prediction)

  0%|          | 0/45 [00:00<?, ?it/s]

(4, 32, 224, 224, 3)


  2%|▏         | 1/45 [00:50<37:22, 50.97s/it]

(4, 32, 224, 224, 3)


  4%|▍         | 2/45 [01:25<29:37, 41.33s/it]

...

 96%|█████████▌| 43/45 [25:21<01:07, 33.75s/it]

(5, 32, 224, 224, 3)


 98%|█████████▊| 44/45 [26:00<00:35, 35.43s/it]

(5, 32, 224, 224, 3)


100%|██████████| 45/45 [26:41<00:00, 35.59s/it]


In [ ]:
sample_submission = pd.read_csv(src_path + '/sample_submission.csv')
sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv(src_path+'/slow_fast_3.csv', index=False)

,file_path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5
0,./test\file_142,0.000838,0.001732,0.001823,0.143321,0.049080,0.803207
1,./test\file_143,0.143214,0.018724,0.004978,0.002807,0.473080,0.357197
2,./test\file_144,0.006091,0.079419,0.126392,0.770817,0.005787,0.011493
3,./test\file_145,0.000145,0.000099,0.306643,0.015350,0.020308,0.657454
4,./test\file_146,0.227987,0.008058,0.008015,0.020681,0.008653,0.726606
